# mission
### 코드 탭에서 "new notebook"으로 문제 풀기 (생성된 csv파일 다운로드)
### 예측한 csv파일 "Submit Predictions"하기
[분류] 여행 보험 패키지 상품을 구매할 확률 값을 구하시오 / 리더보드 제출용
* 예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
* 평가: roc-auc 평가지표

### data: t2-1-train.csv, t2-1-test.csv

### 제출 형식
```
id,TravelInsurance
0,0.3
1,0.48
2,0.3
3,0.83
```

In [1]:
# EDA
import pandas as pd
x_train = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-1-train.csv')
# print(x_train.info())
# AnnualIncome에 4개의 결측치 존재해서 확인 후 처리
# print(x_train['AnnualIncome'].describe())
# 평균으로 처리하면 될 듯
train_AnnualIncome_mean = x_train['AnnualIncome'].mean()
x_train['AnnualIncome'].fillna(train_AnnualIncome_mean, inplace = True)

# print(x_train.info())


x_test = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-1-test.csv')
# print(x_test.info())
test_AnnualIncome_mean = x_test['AnnualIncome'].mean()
x_test['AnnualIncome'].fillna(test_AnnualIncome_mean, inplace = True)
x_test_id = x_test.pop('id')

# x_train.nunique()
# object의 unique 값이 모두 2개씩밖에 없다.
# 그냥 LabelEncoding 하기로 결정

# corr 확인
# x_train.corr()

# x_train['id'] 삭제하기
x_train.drop(columns = ['id'], inplace = True)

# LabelEncoding하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['Employment Type'] = encoder.fit_transform(x_train['Employment Type'])
x_train['GraduateOrNot'] = encoder.fit_transform(x_train['GraduateOrNot'])
x_train['FrequentFlyer'] = encoder.fit_transform(x_train['FrequentFlyer'])
x_train['EverTravelledAbroad'] = encoder.fit_transform(x_train['EverTravelledAbroad'])

x_test['Employment Type'] = encoder.fit_transform(x_test['Employment Type'])
x_test['GraduateOrNot'] = encoder.fit_transform(x_test['GraduateOrNot'])
x_test['FrequentFlyer'] = encoder.fit_transform(x_test['FrequentFlyer'])
x_test['EverTravelledAbroad'] = encoder.fit_transform(x_test['EverTravelledAbroad'])

# y_train 만들기
y_train = x_train.pop('TravelInsurance')
y_train

# 스케일링하기
# # 1. standard scaling
# from sklearn.preprocessing import StandardScaler
# scaler1 = StandardScaler()

# x_train = pd.DataFrame(scaler1.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler1.fit_transform(x_test), columns = x_test.columns)
# # x_train.corr()

# 2. Robust scaling
from sklearn.preprocessing import RobustScaler
scaler2 = RobustScaler()
x_train = pd.DataFrame(scaler2.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler2.fit_transform(x_test), columns = x_test.columns)

# 1. 의사결정나무 
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
model.fit(x_train, y_train)
y_test = pd.DataFrame(model.predict_proba(x_test))
result = pd.concat([x_test_id, y_test], axis = 1)
result.columns = ['id', '1', 'TravelInsurance']
result = result[['id', 'TravelInsurance']]
result.to_csv('Submission.csv', index = False)
result

# score : 0.81135

# # 2. 랜덤포레스트
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(bootstrap=True, max_depth=10)
# model.fit(x_train, y_train)

# y_test = pd.DataFrame(model.predict_proba(x_test))
# result = pd.concat([x_test_id, y_test], axis = 1)
# result.columns = ['id', '1', 'TravelInsurance']
# result = result[['id', 'TravelInsurance']]
# result.to_csv('Submission.csv', index = False)
# result

# score : 0.79705

,id,TravelInsurance
0,0,0.330579
1,1,0.171134
2,2,0.074627
3,3,0.978947
4,4,0.000000
...,...,...
492,492,0.171134
493,493,0.915663
494,494,0.330579
495,495,0.978947
